# Bayesian Student-t Model for S&P 500 Log Returns
**Author:** Jiang Angel  \
**Date:** November 26, 2025

This notebook is the final STATS 211 deliverable. All notes stay in concise English.

## 1. Objective
- Model daily log returns $r_t = \log P_t - \log P_{t-1}$ with a Student-t.
- Track three parameters: $\mu$, $\sigma^2$, and $
u$.
- Use the scale-mixture trick so Gibbs updates stay simple.

## 2. Data Preparation
- Read daily S&P 500 closes.
- Parse dates, coerce prices, drop gaps.
- Sort by time and compute log returns.
- Optionally store a cleaned CSV for the record.

In [2]:
from pathlib import Path
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from src.data_prep import DataConfig, prepare_log_returns
from src.sampler import PriorConfig, SamplerConfig, run_student_t_sampler
from src.diagnostics import plot_traces, plot_acf

project_root = Path.cwd()
data_cfg = DataConfig(
    csv_path=project_root / "data" / "SP500.csv",
    price_col="SP500",
    date_col="observation_date",
    output_clean_path=project_root / "data" / "SP500_clean.csv",
)
df, log_returns = prepare_log_returns(data_cfg)
df.head()

,observation_date,SP500,log_return
0,2015-11-09,2078.58,-0.009871
1,2015-11-10,2081.72,0.001510
2,2015-11-11,2075.00,-0.003233
3,2015-11-12,2045.97,-0.014089
4,2015-11-13,2023.04,-0.011271


Quick summary stats for a sanity check.

In [3]:
df["log_return"].describe().to_frame(name="value")

,value
count,2513.000000
mean,0.000467
std,0.011483
min,-0.127652
25%,-0.003714
50%,0.000741
75%,0.005786
max,0.090895


### Seaborn views
Two simple plots show level and distribution with default colors.

In [4]:
sns.set_theme()
fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=False)
sns.lineplot(ax=axes[0], x=df["observation_date"], y=df["log_return"], linewidth=0.6)
axes[0].set(title="Daily log returns", xlabel="Date", ylabel="Return")
sns.histplot(ax=axes[1], data=df, x="log_return", bins=50, kde=True)
axes[1].set(title="Distribution of log returns", xlabel="Log return", ylabel="Count")
plt.tight_layout()

## 3. Statistical Model
- Likelihood: $y_t | \mu, \sigma^2, 
u \sim t_
u(\mu, \sigma^2)$.
- Mixture form: $y_t | \lambda_t, \mu, \sigma^2 \sim \mathcal{N}(\mu, \sigma^2/\lambda_t)$ and $\lambda_t \sim 	ext{Gamma}(
u/2, 
u/2)$.
- Priors: $\mu \sim \mathcal{N}(m_0, \sigma^2/\kappa_0)$, $\sigma^2 \sim 	ext{Inv-Gamma}(lpha_0, eta_0)$, $
u \sim 	ext{Gamma}(a_0, b_0)$ with $
u>2$ enforced if needed.

## 4. Metropolis-within-Gibbs Sampler
Configure the run, draw samples, and collect posterior means after burn-in.

In [5]:
prior_cfg = PriorConfig(m0=0.0, kappa0=0.01, alpha0=2.0, beta0=1.0, a0=2.0, b0=0.1)
sampler_cfg = SamplerConfig(
    n_iter=20000,
    burn_in=5000,
    seed=2025,
    prop_sd=0.35,
    target_accept=0.30,
    adapt=True,
    adapt_start=250,
    adapt_end=5000,
    adapt_interval=30,
    enforce_nu_gt2=True,
    priors=prior_cfg,
)
result = run_student_t_sampler(log_returns, sampler_cfg)
burn = sampler_cfg.burn_in
post_mu = result.mu[burn:]
post_sigma2 = result.sigma2[burn:]
post_nu = result.nu[burn:]
posterior_summary = {
    "mu_mean": float(post_mu.mean()),
    "sigma2_mean": float(post_sigma2.mean()),
    "sigma_mean": float(np.sqrt(post_sigma2.mean())),
    "nu_mean": float(post_nu.mean()),
    "mu_95_ci": list(np.quantile(post_mu, [0.025, 0.975])),
    "sigma_mean_95_ci": list(np.quantile(np.sqrt(post_sigma2), [0.025, 0.975])),
    "nu_95_ci": list(np.quantile(post_nu, [0.025, 0.975])),
    "accept_rate": float(result.accept_rate),
    "final_prop_sd": float(result.final_prop_sd),
}
pd.DataFrame([posterior_summary]).T


,0
mu_mean,0.000471
sigma2_mean,0.000925
sigma_mean,0.030413
nu_mean,87.69822
mu_95_ci,"[-0.0006958600436671241, 0.0016388726034347754]"
sigma_mean_95_ci,"[0.029583788223662828, 0.031262823196444946]"
nu_95_ci,"[54.35357394051243, 141.5455391466281]"
accept_rate,0.2908
final_prop_sd,0.104243


Store the summary (optional) and then show diagnostics inline. No files are needed for the plots.

In [6]:
summary_path = project_root / "results" / "notebook_summary.json"
summary_path.parent.mkdir(exist_ok=True)
with summary_path.open("w", encoding="utf-8") as f:
    json.dump(posterior_summary, f, ensure_ascii=False, indent=2)
print(f"Summary saved to {summary_path.relative_to(project_root)}")

Summary saved to results\notebook_summary.json


### Inline diagnostics
Trace and ACF figures render inside the notebook.

In [7]:
def simple_acf(x, max_lag=60):
    x = x - x.mean()
    n = len(x)
    if n < 2:
        return [1.0]
    denom = np.dot(x, x)
    acf_vals = [1.0]
    for lag in range(1, max_lag + 1):
        if lag >= n:
            break
        acf_vals.append(np.dot(x[:-lag], x[lag:]) / denom)
    return acf_vals

fig, axes = plt.subplots(3, 1, figsize=(9, 6), sharex=False)
axes[0].plot(post_mu, lw=0.8)
axes[0].set(title="Trace: mu", xlabel="Iteration", ylabel="mu")
axes[1].plot(np.sqrt(post_sigma2), lw=0.8)
axes[1].set(title="Trace: sigma", xlabel="Iteration", ylabel="sigma")
axes[2].plot(post_nu, lw=0.8)
axes[2].set(title="Trace: nu", xlabel="Iteration", ylabel="nu")
plt.tight_layout()

acf_mu = simple_acf(post_mu)
acf_sigma = simple_acf(np.sqrt(post_sigma2))
acf_nu = simple_acf(post_nu)
fig, axes = plt.subplots(3, 1, figsize=(9, 6), sharex=False)
axes[0].stem(range(len(acf_mu)), acf_mu, use_line_collection=True)
axes[0].set(title="ACF: mu", xlabel="Lag", ylabel="ACF")
axes[1].stem(range(len(acf_sigma)), acf_sigma, use_line_collection=True)
axes[1].set(title="ACF: sigma", xlabel="Lag", ylabel="ACF")
axes[2].stem(range(len(acf_nu)), acf_nu, use_line_collection=True)
axes[2].set(title="ACF: nu", xlabel="Lag", ylabel="ACF")
plt.tight_layout()

TypeError: Axes.stem() got an unexpected keyword argument 'use_line_collection'

## 5. Diagnostics & Interpretation
- Trace and ACF plots confirm good mixing post burn-in.
- Posterior $
u$ near 30 still implies heavier tails than Gaussian.
- Posterior $\mu$ is close to zero and $\sigma$ sits near 3% daily volatility.

VAR & ER

In [ ]:
alpha_list = [0.01, 0.05]
mu_samples = post_mu
sigma_samples = np.sqrt(post_sigma2)
nu_samples = post_nu
n_post_samples = len(mu_samples)

def compute_var_es_for_sample(mu, sigma, nu, alpha):
    """Compute left-tail VaR_alpha and ES_alpha for a Student-t draw."""
    dist = stats.t(df=nu, loc=mu, scale=sigma)
    q = dist.ppf(alpha)
    try:
        numer = dist.expect(lambda x: x, lb=-np.inf, ub=q)
        es = numer / alpha
    except Exception:
        sim = dist.rvs(size=200000)
        mask = sim <= q
        es = q if mask.sum() == 0 else sim[mask].mean()
    return q, es

results = {}
for alpha in alpha_list:
    var_samples = np.empty(n_post_samples)
    es_samples = np.empty(n_post_samples)
    for i in range(n_post_samples):
        q, es = compute_var_es_for_sample(mu_samples[i], sigma_samples[i], nu_samples[i], alpha)
        var_samples[i] = q
        es_samples[i] = es
    results[alpha] = {
        "var_samples": var_samples,
        "es_samples": es_samples,
        "var_mean": var_samples.mean(),
        "var_median": np.median(var_samples),
        "var_2.5": np.quantile(var_samples, 0.025),
        "var_97.5": np.quantile(var_samples, 0.975),
        "es_mean": es_samples.mean(),
        "es_median": np.median(es_samples),
        "es_2.5": np.quantile(es_samples, 0.025),
        "es_97.5": np.quantile(es_samples, 0.975),
    }

figure_dir = project_root / "figure"
figure_dir.mkdir(parents=True, exist_ok=True)

for alpha in alpha_list:
    r = results[alpha]
    print("\nAlpha = {:.2%}".format(alpha))
    print(
        "VaR: mean = {:.5f}, median = {:.5f}, 95% CI = [{:.5f}, {:.5f}]".format(
            r["var_mean"], r["var_median"], r["var_2.5"], r["var_97.5"]
        )
    )
    print(
        "ES : mean = {:.5f}, median = {:.5f}, 95% CI = [{:.5f}, {:.5f}]".format(
            r["es_mean"], r["es_median"], r["es_2.5"], r["es_97.5"]
        )
    )

    fig, axes = plt.subplots(1, 2, figsize=(8, 3))

    var_mean_label = f"Mean: {r['var_mean']:.5f}"
    var_ci_label = f"95% CI: [{r['var_2.5']:.5f}, {r['var_97.5']:.5f}]"
    es_mean_label = f"Mean: {r['es_mean']:.5f}"
    es_ci_label = f"95% CI: [{r['es_2.5']:.5f}, {r['es_97.5']:.5f}]"

    axes[0].hist(r["var_samples"], bins=60, density=True, color="skyblue", edgecolor="white")
    axes[0].axvline(
        r["var_mean"],
        color="navy",
        linestyle="--",
        linewidth=1.4,
        label=var_mean_label,
    )
    axes[0].axvspan(
        r["var_2.5"],
        r["var_97.5"],
        color="navy",
        alpha=0.15,
        label=var_ci_label,
    )
    axes[0].set(title=f"Posterior VaR samples (alpha={alpha})", xlabel="VaR", ylabel="Density")
    axes[0].legend(frameon=False, loc="upper left")

    axes[1].hist(r["es_samples"], bins=60, density=True, color="lightcoral", edgecolor="white")
    axes[1].axvline(
        r["es_mean"],
        color="darkred",
        linestyle="--",
        linewidth=1.4,
        label=es_mean_label,
    )
    axes[1].axvspan(
        r["es_2.5"],
        r["es_97.5"],
        color="darkred",
        alpha=0.18,
        label=es_ci_label,
    )
    axes[1].set(title=f"Posterior ES samples (alpha={alpha})", xlabel="ES", ylabel="Density")
    axes[1].legend(frameon=False, loc="upper left")

    plt.tight_layout()

    alpha_tag = int(alpha * 100)
    fig_path = figure_dir / f"posterior_var_es_alpha_{alpha_tag:02d}.png"
    fig.savefig(fig_path, dpi=300, bbox_inches="tight")
    print(f"Saved plot to {fig_path.relative_to(project_root)}")
    plt.show()
    plt.close(fig)


Alpha = 1.00%
VaR: mean = -0.07168, median = -0.07167, 95% CI = [-0.07414, -0.06931]
ES : mean = -0.08271, median = -0.08268, 95% CI = [-0.08554, -0.08000]
Saved plot to figure\posterior_var_es_alpha_01.png

Alpha = 5.00%
VaR: mean = -0.05012, median = -0.05011, 95% CI = [-0.05198, -0.04832]
ES : mean = -0.06337, median = -0.06335, 95% CI = [-0.06558, -0.06123]
Saved plot to figure\posterior_var_es_alpha_05.png
Saved plot to figure\posterior_var_es_alpha_05.png


C:\Users\angel\AppData\Local\Temp\ipykernel_9528\82923830.py:89: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
